<a href="https://colab.research.google.com/github/wasilone11/grroom/blob/main/Copy_of_tiny_merged_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from IPython.display import HTML
from subprocess import getoutput
s = getoutput('nvidia-smi')
print(s)
if 'K80' in s:
    gpu = 'K80'
elif 'T4' in s:
    gpu = 'T4'
elif 'P100' in s:
    gpu = 'P100'
else:
    gpu='DONT PROCEED'
display(HTML(f"<h1>{gpu}</h1>"))

Sat May  8 12:15:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [4]:
%cd /gdrive/MyDrive/darknet
!chmod +x ./darknet
!./darknet

/gdrive/MyDrive/darknet
usage: ./darknet <function>


In [5]:
import os
import random
folder="topwear_merged"
category="t_neckline_a1"
if category=="":
  inputpath="../"+folder
  opPath=folder+"_tiny_model"
else:
  inputpath="../"+folder+'/'+category
  opPath=folder+"_"+category+"_tiny_model"
if opPath not in os.listdir():
  !mkdir $opPath

In [ ]:
d={}
for i in [m for m in os.listdir(inputpath) if "." not in m]:
 
    all_temp=[]
    for j in [m for m in os.listdir(inputpath+"/"+i) if "." not in m]:
        temp=[]
        for k in [m for m in os.listdir(inputpath+"/"+i+"/"+j) if "." not in m]:
            tmp_lst=[inputpath+'/'+i+'/'+j+'/'+k+'/'+m for m in os.listdir(inputpath+'/'+i+'/'+j+'/'+k)]
            temp.extend(tmp_lst)
 
        temp=[m for m in temp if (m[:3]!="ls:")&("classes" not in m)]
        temp.sort()
        images=[m for m in temp if ".txt" not in m]
        txts=[m for m in temp if ".txt" in m]
        all_temp.extend(temp)
        print(folder,i,j,len(images),len(txts))
    d[i]=all_temp

topwear_merged boat choli 106 106
topwear_merged boat kurta 106 106
topwear_merged button_down choli 106 106
topwear_merged button_down kurta 106 106
topwear_merged keyhole choli 106 106
topwear_merged keyhole kurta 106 106
topwear_merged mandarin choli 106 106
topwear_merged mandarin kurta 106 106
topwear_merged one_shoulder choli 212 212
topwear_merged round choli 106 106
topwear_merged round kurta 106 106
topwear_merged scoop choli 212 212
topwear_merged shoulder_off choli 212 212
topwear_merged square choli 106 106
topwear_merged square kurta 106 106
topwear_merged sweetheart_neck choli 212 212
topwear_merged v_neck choli 106 106
topwear_merged v_neck kurta 106 106


In [ ]:
tmp=list(d.keys())
for i in tmp:
    print(i,len(d[i]))

boat 424
button_down 424
keyhole 424
mandarin 424
one_shoulder 424
round 424
scoop 424
shoulder_off 424
square 424
sweetheart_neck 424
v_neck 424


In [ ]:
train=[]
for i in d.keys():
    for j in range(len(d[i])-1):
        if d[i][j][:d[i][j].rfind('.')]==d[i][j+1][:d[i][j+1].rfind('.')]:
            train.append(d[i][j])
            j+=1

In [ ]:
count=0
classes={}
for i in d.keys():
    classes[count]=i
    for j in d[i]:
        if ".txt" in j:
            f=open(j,'r+')
            s=[k.split(" ") for k in f.read().split('\n')[:-1]]
            s=[[str(count)]+k[1:] for k in s]+[['']]
            s="\n".join([" ".join(k) for k in s])
            f.seek(0)
            f.write(s)
            f.close()
    count+=1

In [ ]:
f=open(inputpath+"/classes.txt",'w')
c=[]
for i in range(len(classes)):
    c.append(classes[i])
f.write("\n".join(c))
f.close()

In [ ]:
!cat $inputpath/classes.txt

boat
button_down
keyhole
mandarin
one_shoulder
round
scoop
shoulder_off
square
sweetheart_neck
v_neck

In [ ]:
classesPath=inputpath+"/classes.txt"
f=open(classesPath,"r")
names=f.read()
f.close()
f=open(inputpath+"/obj.names","w")
f.write(names)
f.close()

In [ ]:
numClass=len(names.split("\n"))
objdata=["classes= "+str(numClass),
         "train = "+inputpath+"/train.txt",
         "valid = "+inputpath+"/valid.txt",
         "names = "+inputpath+"/obj.names",
         "backup = "+opPath+"/"]
f=open(inputpath+"/obj.data","w")
f.write("\n".join(objdata))
f.close()

In [ ]:
train=[i for i in train if ".txt" not in i]
train=[i for i in train if ".gif" not in i]
train=[i for i in train if ".jfif" not in i]
random.shuffle(train)
l=len(train)
valid=train[int(l*5/6):]
train=train[:int(l*5/6)]
f=open(inputpath+'/train.txt','w')
f.write("\n".join(train))
f.close()
f=open(inputpath+'/valid.txt','w')
f.write("\n".join(valid))
f.close()

In [ ]:
numClass

11

In [ ]:
if "yolov4-tiny-custom.cfg" not in os.listdir(inputpath):
    !cp cfg/yolov4-tiny-custom.cfg $inputpath/yolov4-tiny-custom.cfg
else:
    !rm $inputpath/yolov4-tiny-custom.cfg
    !cp cfg/yolov4-tiny-custom.cfg $inputpath/yolov4-tiny-custom.cfg

In [ ]:
#change number of class and filters
comm='s/classes=80/classes='+str(numClass)+"/"
!sed -i $comm $inputpath/yolov4-tiny-custom.cfg
comm='s/filters=255/filters='+str((numClass+5)*3)+"/"
!sed -i $comm $inputpath/yolov4-tiny-custom.cfg

#change number of steps and max batches
max_batches=max(6000,(numClass)*2000)
comm='s/max_batches\ =\ 500200/max_batches\ =\ '+str(max_batches)+"/"
!sed -i $comm $inputpath/yolov4-tiny-custom.cfg
comm='s/steps=400000,450000/steps='+str(int(0.8*max_batches))+','+str(int(0.9*max_batches))+"/"
!sed -i $comm $inputpath/yolov4-tiny-custom.cfg

netSize=416  #can be multiple of 32 or 608(but makes training slow)
comm='s/width=480/width='+str(netSize)+"/"
!sed -i $comm $inputpath/yolov4-tiny-custom.cfg
comm='s/height=480/height='+str(netSize)+"/"
!sed -i $comm $inputpath/yolov4-tiny-custom.cfg

batch=64
subdiv=1
comm='s/batch=64/batch='+str(batch)+'/'
!sed -i $comm $inputpath/yolov4-tiny-custom.cfg
comm='s/subdivisions=16/subdivisions='+str(subdiv)+'/'
!sed -i $comm $inputpath/yolov4-tiny-custom.cfg

In [ ]:
if "yolov4.conv.137" not in os.listdir():
    !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137
if "yolov4-tiny.weights" not in os.listdir():
    !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights
if "yolov4-tiny.conv.29" not in os.listdir():
    !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29

In [ ]:
!./darknet detector train $inputpath/obj.data $inputpath/yolov4-tiny-custom.cfg yolov4-tiny.conv.29 -dont_show -map

Streaming output truncated to the last 5000 lines.
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.805714), count: 40, class_loss = 0.080263, iou_loss = 3.442658, total_loss = 3.522921 
 total_bbox = 1168839, rewritten_bbox = 0.019849 % 

 (next mAP calculation at 17819 iterations) 
 Last accuracy mAP@0.5 = 51.95 %, best = 51.95 % 
 17769: 0.099957, 0.092367 avg loss, 0.000261 rate, 1.056504 seconds, 1137216 images, 2.032249 hours left
Loaded: 0.519095 seconds - performance bottleneck on CPU or Disk HDD/SSD
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.779659), count: 32, class_loss = 0.115746, iou_loss = 0.442510, total_loss = 0.558255 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.734624), count: 36, class_loss = 0.138818, iou_loss = 3.396868, total_loss = 3.535686 
 total_bbox = 1168907, rewritten_bbox = 0.019848 % 

 (next mAP calculation at 17819 iterations) 
 Last accuracy mA

In [6]:
!./darknet detector train $inputpath/obj.data $inputpath/yolov4-tiny-custom.cfg $opPath/yolov4-tiny-custom_last.weights -dont_show -map

Streaming output truncated to the last 5000 lines.
 total_bbox = 211591, rewritten_bbox = 0.023630 % 

 (next mAP calculation at 21567 iterations) 

 Tensor Cores are used.
 Last accuracy mAP@0.5 = 54.30 %, best = 55.57 % 
 21517: 0.085795, 0.081453 avg loss, 0.000026 rate, 0.653170 seconds, 1377088 images, 0.208676 hours left
Loaded: 0.545816 seconds - performance bottleneck on CPU or Disk HDD/SSD
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.817066), count: 31, class_loss = 0.072569, iou_loss = 0.552232, total_loss = 0.624802 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.812267), count: 31, class_loss = 0.039835, iou_loss = 2.910100, total_loss = 2.949934 
 total_bbox = 211653, rewritten_bbox = 0.023624 % 

 (next mAP calculation at 21567 iterations) 

 Tensor Cores are used.
 Last accuracy mAP@0.5 = 54.30 %, best = 55.57 % 
 21518: 0.056405, 0.078948 avg loss, 0.000026 rate, 0.586408 seconds, 1377152 images, 0.

In [7]:
!./darknet detector map $inputpath/obj.data $inputpath/yolov4-tiny-custom.cfg $opPath/yolov4-tiny-custom_best.weights

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 1, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 2    416 x 416 x   3 ->  208 x 208 x  32 0.075 BF
   1 conv     64       3 x 3/ 2    208 x 208 x  32 ->  104 x 104 x  64 0.399 BF
   2 conv     64       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.797 BF
   3 route  2 		                       1/2 ->  104 x 104 x  32 
   4 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   5 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   6 route  5 4 	                           ->  104 x 104 x  64 
   7 conv     64       1 x 1/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.089 BF
   8 route  2 7 	                           ->  104 x 104 x 128 
   9 